# Testing

In [14]:
import utils
import functions

import openai
import pandas as pd
from random import randrange
from math import exp
from dotenv import load_dotenv
import os

# 개인 api key
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

m_num = 5
"""
- `curie:ft-personal-2023-05-31 08-28-50` : csi 데이터 학습
- `curie:ft-personal-2023-06-11 18-10-30` : 總 데이터 사고경위만 학습
- `curie:ft-personal-2023-06-12-06-34-37` : 總 데이터 키워드化하여 사고경위와 함께 학습
- `curie:ft-personal-2023-07-14-07-12-02` : Structured + Unstructured
- `curie:ft-personal-2023-07-15-11-00-14` : Unstructured only
- `curie:ft-personal-2023-08-05-23-22-50` : Causes
"""
ft_models = {1 : "curie:ft-personal-2023-05-31 08-28-50",
            2 : "curie:ft-personal-2023-06-11 18-10-30",
            3 : "curie:ft-personal-2023-06-12-06-34-37",
            4 : "curie:ft-personal-2023-07-14-07-12-02",
            5 : "curie:ft-personal-2023-07-15-11-00-14",
            6 : "curie:ft-personal-2023-08-05-23-22-50"}
pp_datas = {1 : "C:/Users/bhyoo/Desktop/Causual Analysis/CSI_DESKTOP_Test/s_filtered_sieben_codex_v3_prepared.jsonl",
            2 : "C:/Users/bhyoo/Documents/My Projects/Causal Analysis/GPT Fine-tuning/prepocess/prepocessed.jsonl",
            3 : "C:/Users/bhyoo/Documents/My Projects/Causal Analysis/GPT Fine-tuning/prepocess/prepocessed with keywords.jsonl",
            4 : "C:/Users/bhyoo/Documents/My Projects/Causal Analysis/GPT Fine-tuning/prepocess/result35.jsonl",
            5 : "C:/Users/bhyoo/Documents/My Projects/Causal Analysis/GPT Fine-tuning/prepocess/result37.jsonl",
            6 : "C:/Users/bhyoo/Documents/My Projects/Causal Analysis/GPT Fine-tuning/prepocess/causes.jsonl"}
ft_model = ft_models[m_num]
pp_data = pp_datas[m_num]
classes = ["기타", 
            "넘어짐", 
            "부딪힘", 
            "절단, 베임, 찔림",
            "추락", 
            "협착"]

Model = functions.FineTuneModel(OPENAI_API_KEY, ft_model, classes)

## Data Load

In [15]:
no = randrange(1, 5_000)
# no = 0

sample_data = pd.read_json(pp_data, lines=True).iloc[no]
sample_prompt = sample_data['prompt'] + "\n\n###\n\n"
sample_completion = sample_data['completion']

print(f"[row number] {no}\n[prompt] {sample_prompt}\n[completion] {sample_completion}")

[row number] 2875
[prompt] ① 언제 : 2021년 6월 15일 (火) 07시 40분경

② 어디서: 화도졸음쉼터 양양방향(sta.0+948지점)차량 진출입로 입구에서

③ 누가 : 장비 전담유도원 ***가

④ 무엇을 : 장비 유도를 위해 현장 진출입로에서 신호 업무를 보던 중

⑤ 어떻게 : 살수차량이 살수를 마치고 운전원 ***가 후진을 하다가

⑥ 왜 : 후측방을 주시 하지 못하여 부딪힘 사고 발생

###


[completion] 부딪힘


In [16]:
pred, probs = Model.run(prompt=sample_prompt)
print(f"[prediction] {pred}\n[probability] {probs*100:.2f}%")

[prediction] 부딪힘
[probability] 96.69%


## Prompt

In [31]:
"""Prompt 작성"""
# sample_prompt = '''개인 지병에 의한 병사이며2019.07.22. 월요일 오전 11시 30분경 설비 보온재 시공 근로자 OOO씨가 106동 지하1층 주차장에서이동식 틀비계를 끌어안고 있는 것을 동료 근로자 OOO씨가 발견하고 “왜그래? 왜그래?”라고 물으니OOO씨가 “기운이 없어”라고 답하며 주저 앉으려고 하자 조한월씨가 부축하여 바닥에 누인 다음119로 부천 카톨릭 성모병원으로 이송하였고부천 카톨릭 성모병원에서 뇌출혈 진단을 받고 2019.07.22 당일 수술 후 중환자실에 입원중2019.08.04 오전 07시 36분에 병사한 것임.날씨 맑음 기온 31℃ 습도 74%'''
# 정답 : 기타

'Prompt 작성'

## Completion

In [118]:
"""답만 보는 코드"""
res = openai.Completion.create(model=ft_model, prompt=sample_prompt, max_tokens=50, temperature=0.2, logprobs=6)
res['choices'][0]['text'].lstrip(' ').split(' ')[0]

'넘어짐\n넘어짐\n부딪힘\n골절\n계절\n\n기'

In [119]:
res['choices'][0]['logprobs']['top_logprobs'][0]

<OpenAIObject at 0x28c39fe82c0> JSON: {
  " ": -4.2312784,
  "bytes: \\xe6": -13.923406,
  "bytes: \\xeb": -0.02637415,
  "bytes: \\xec": -4.4664927,
  "bytes:\\xeb": -12.901503
}

In [8]:
"""JSON"""
res

<OpenAIObject text_completion id=cmpl-7c9nFNXaotoZoM3xNIP905AEQUqUB at 0x17823b0ca90> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          227,
          228,
          228,
          228,
          229,
          229,
          229,
          230,
          232,
          233,
          234,
          234,
          234,
          235,
          235,
          235,
          236,
          237,
          238,
          238,
          238,
          239,
          239,
          239,
          240,
          240,
          240,
          241,
          242,
          245,
          246,
          248,
          249,
          250,
          251,
          251,
          251,
          252,
          252,
          252,
          253,
          254,
          254,
          254,
          256,
          257,
          258,
          258,
          258,
          259
        ],
        "token_logprobs": [
 

In [120]:
! openai -k "{OPENAI_API_KEY}" api completions.create -m "{ft_model}" -p "{sample_prompt}"

#사고경위: 2021년 06월 09일 15시경, 110동 306호세대 앞 E/V홀에서 재해자가 보양비닐 제거 작업 중, 약 4~5p 아웃다


In [121]:
neo_target = dict(res['choices'][0]['logprobs']['top_logprobs'][0])
"""
<OpenAIObject at 0x1bc978939f0> JSON: {
    " ": -6.1572285,
    "bytes: \\xeb": -7.2011843,
    "bytes: \\xec": -0.0028715432,
    "bytes:\\xec": -13.375288,
    "bytes:\\xec\\x9d": -13.66861
}
"""
print(neo_target)
# neo_target.to_dict()
for n, w in neo_target.items():
    print(n, exp(w), '%')

{'bytes: \\xe6': -13.923406, 'bytes:\\xeb': -12.901503, ' ': -4.2312784, 'bytes: \\xec': -4.4664927, 'bytes: \\xeb': -0.02637415}
bytes: \xe6 8.977214554513625e-07 %
bytes:\xeb 2.4942985763728727e-06 %
  0.014533798658806945 %
bytes: \xec 0.011487535511367448 %
bytes: \xeb 0.9739706103241863 %


In [54]:
from math import exp

print(b"\\xeb".decode('utf-8'), exp(-0.0028715432))

\xeb 0.9971325757366633
